<a href="https://colab.research.google.com/github/FanusArefaine/Apache-Spark-Essentials/blob/master/Spark_Transformations_and_Actions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-bin-hadoop2.7.tgz
!pip install -q findspark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [12]:
!ls /usr/lib/jvm/

default-java		   java-11-openjdk-amd64     java-8-openjdk-amd64
java-1.11.0-openjdk-amd64  java-1.8.0-openjdk-amd64


In [13]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"

In [15]:
import findspark
findspark.init()
from pyspark import SparkContext

sc = SparkContext.getOrCreate()
sc


<SparkContext master=local[*] appName=pyspark-shell>

In [16]:
import pyspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [17]:
import sys, tempfile, urllib 
from pyspark.sql.functions import *

In [18]:
#setting up the base directory 

BASE_DIR = '/tmp'
CORONA_DATA_FILE = os.path.join(BASE_DIR, 'corona_data.csv')

In [19]:
import urllib.request
corona_data = urllib.request.urlretrieve('https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/coronavirus/corona_dataset_latest.csv', CORONA_DATA_FILE)

In [20]:
BASE_DIR = '/tmp'
TWITTER_DATA_FILE = os.path.join(BASE_DIR, 'twitter_data.csv')

In [21]:
tweet_data = urllib.request.urlretrieve('https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/coronavirus/tweets.csv', TWITTER_DATA_FILE)

In [22]:
!ls /tmp

blockmgr-27545475-8797-43ab-92d7-052a36ea3e27
corona_data.csv
dap_multiplexer.f35098dd5be8.root.log.INFO.20210316-033839.47
dap_multiplexer.INFO
debugger_2qh9vzw9kd
drivefs_ipc.0
drivefs_ipc.0_shell
hsperfdata_root
initgoogle_syslog_dir.0
spark-651cdfa4-c21b-42be-b1d4-0bafd604bbff
spark-ae2db41d-5266-457a-9601-6d6c66743572
tmpx4qlsvjw
twitter_data.csv


In [23]:
corona_df = spark.read.option("inferSchema", "true").csv("/tmp/corona_data.csv", header=True)

In [24]:
corona_df.show()

+---+----------------+--------------------+--------+---------+----------+---------+-----+---------+--------------------+----+
|_c0|           State|             Country|     Lat|     Long|      Date|Confirmed|Death|Recovered|       state_cleaned|City|
+---+----------------+--------------------+--------+---------+----------+---------+-----+---------+--------------------+----+
|  0|            null|            Thailand|    15.0|    101.0|2020-01-22|        2|    0|        0|             Bangkok|null|
|  1|            null|               Japan|    36.0|    138.0|2020-01-22|        2|    0|        0|             Hiraide|null|
|  2|            null|           Singapore|  1.2833| 103.8333|2020-01-22|        0|    0|        0|           Singapore|null|
|  3|            null|               Nepal| 28.1667|    84.25|2020-01-22|        0|    0|        0|           Kathmandu|null|
|  4|            null|            Malaysia|     2.5|    112.5|2020-01-22|        0|    0|        0|             Sarawa

In [25]:
corona_df.count()

28143

In [26]:
# importing tweeter data 

twitter_df = spark.read.option("inferSchema", "true").csv("/tmp/twitter_data.csv", header=True)

In [27]:
twitter_df.show()

+---+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|_c0| geo|                text|                user|            location|            entities|           sentiment|             country|
+---+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  0|null|What is God sayin...|          petodinice|               Lagos|[('about #', 'CAR...|{'neg': 0.0, 'neu...|             Nigeria|
|  1|null|"BREAKING: ""this...| but i took the t...|                -… "|     JerryfranksonJF|      Abuja, Nigeria|"[(""Arsenal's Mi...|
|  2|null| #Coronavirus tes...|              cek422|   Pennsylvania, USA|                  []|{'neg': 0.173, 'n...|                 USA|
|  3|null| Get ready for ma...|        InfectiousDz|                 NYC|[('World', 'ORG')...|{'neg': 0.085, 'n...|                 USA|
|  4|null| The #coronavirus...|          

### *Transformations and Actions*

In [28]:
# Transformation creates an execution plan --> outputs a dataframe 

twitter_df.filter("country='USA'")

DataFrame[_c0: int, geo: string, text: string, user: string, location: string, entities: string, sentiment: string, country: string]

In [29]:
twitter_df.filter('country="USA"').show()

+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|_c0| geo|                text|           user|            location|            entities|           sentiment|country|
+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|  2|null| #Coronavirus tes...|         cek422|   Pennsylvania, USA|                  []|{'neg': 0.173, 'n...|    USA|
|  3|null| Get ready for ma...|   InfectiousDz|                 NYC|[('World', 'ORG')...|{'neg': 0.085, 'n...|    USA|
|  5|null| COVID-19 update ...| StewartNgilana|Durban | Port Eli...|[('Italy', 'GPE')...|{'neg': 0.178, 'n...|    USA|
|  6|null| It’s painful to ...|      BWheatnyc|             Florida|                  []|{'neg': 0.098, 'n...|    USA|
|  8|null| Questions about ...|    straightj23|        Columbus, OH|[('NAfME', 'CARDI...|{'neg': 0.0, 'neu...|    USA|
| 13|null| We’re the heck w...| harrytiffanyiv| 

In [30]:
twitter_df.filter("country='USA' and location like '%New%'").show()

+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|_c0| geo|                text|           user|            location|            entities|           sentiment|country|
+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
| 31|null| I ordered Alex J...|       rcgillan|       New York, USA|[('Alex Jones', '...|{'neg': 0.109, 'n...|    USA|
| 49|null| This week we are...|  JamesWithers3|  New York, New York|[('This week', 'D...|{'neg': 0.0, 'neu...|    USA|
| 62|null|Fear will kill #C...| RobertPPurcell|norther New Jerse...|                ['']|{'neg': 0.798, 'n...|    USA|
|228|null|This is a very co...|baskingntheGlow|       New York City|[('hourly', 'TIME')]|{'neg': 0.12, 'ne...|    USA|
|238|null|I’m reposting thi...|   Veronicaromm|     New Jersey, USA|[('English', 'LAN...|{'neg': 0.0, 'neu...|    USA|
|261|null|Too early ...?  #...|      HJeppesen| 

In [31]:
tw_filter_df = twitter_df.filter("country='USA'")

In [32]:
tw_filter_df.explain()

== Physical Plan ==
*(1) Project [_c0#126, geo#127, text#128, user#129, location#130, entities#131, sentiment#132, country#133]
+- *(1) Filter (isnotnull(country#133) AND (country#133 = USA))
   +- FileScan csv [_c0#126,geo#127,text#128,user#129,location#130,entities#131,sentiment#132,country#133] Batched: false, DataFilters: [isnotnull(country#133), (country#133 = USA)], Format: CSV, Location: InMemoryFileIndex[file:/tmp/twitter_data.csv], PartitionFilters: [], PushedFilters: [IsNotNull(country), EqualTo(country,USA)], ReadSchema: struct<_c0:int,geo:string,text:string,user:string,location:string,entities:string,sentiment:strin...




In [33]:
tw_filter_df = twitter_df.filter("country='USA' and location like '%New%'")

In [34]:
tw_filter_df.explain()

== Physical Plan ==
*(1) Project [_c0#126, geo#127, text#128, user#129, location#130, entities#131, sentiment#132, country#133]
+- *(1) Filter (((isnotnull(country#133) AND isnotnull(location#130)) AND (country#133 = USA)) AND Contains(location#130, New))
   +- FileScan csv [_c0#126,geo#127,text#128,user#129,location#130,entities#131,sentiment#132,country#133] Batched: false, DataFilters: [isnotnull(country#133), isnotnull(location#130), (country#133 = USA), Contains(location#130, New)], Format: CSV, Location: InMemoryFileIndex[file:/tmp/twitter_data.csv], PartitionFilters: [], PushedFilters: [IsNotNull(country), IsNotNull(location), EqualTo(country,USA), StringContains(location,New)], ReadSchema: struct<_c0:int,geo:string,text:string,user:string,location:string,entities:string,sentiment:strin...




In [35]:
tw_filter_df.show()

+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|_c0| geo|                text|           user|            location|            entities|           sentiment|country|
+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
| 31|null| I ordered Alex J...|       rcgillan|       New York, USA|[('Alex Jones', '...|{'neg': 0.109, 'n...|    USA|
| 49|null| This week we are...|  JamesWithers3|  New York, New York|[('This week', 'D...|{'neg': 0.0, 'neu...|    USA|
| 62|null|Fear will kill #C...| RobertPPurcell|norther New Jerse...|                ['']|{'neg': 0.798, 'n...|    USA|
|228|null|This is a very co...|baskingntheGlow|       New York City|[('hourly', 'TIME')]|{'neg': 0.12, 'ne...|    USA|
|238|null|I’m reposting thi...|   Veronicaromm|     New Jersey, USA|[('English', 'LAN...|{'neg': 0.0, 'neu...|    USA|
|261|null|Too early ...?  #...|      HJeppesen| 

In [36]:
tw_filter_df = twitter_df.filter(col('location').startswith('N'))

In [37]:
tw_filter_df.explain()

== Physical Plan ==
*(1) Project [_c0#126, geo#127, text#128, user#129, location#130, entities#131, sentiment#132, country#133]
+- *(1) Filter (isnotnull(location#130) AND StartsWith(location#130, N))
   +- FileScan csv [_c0#126,geo#127,text#128,user#129,location#130,entities#131,sentiment#132,country#133] Batched: false, DataFilters: [isnotnull(location#130), StartsWith(location#130, N)], Format: CSV, Location: InMemoryFileIndex[file:/tmp/twitter_data.csv], PartitionFilters: [], PushedFilters: [IsNotNull(location), StringStartsWith(location,N)], ReadSchema: struct<_c0:int,geo:string,text:string,user:string,location:string,entities:string,sentiment:strin...




In [38]:
tw_filter_df.show()

+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------------+
|_c0| geo|                text|           user|            location|            entities|           sentiment|      country|
+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------------+
|  3|null| Get ready for ma...|   InfectiousDz|                 NYC|[('World', 'ORG')...|{'neg': 0.085, 'n...|          USA|
|  7|null| 📽️Friends, I wi...|        LorseaR|New South Wales, ...|[('Twitter', 'GPE...|{'neg': 0.123, 'n...|    Australia|
| 31|null| I ordered Alex J...|       rcgillan|       New York, USA|[('Alex Jones', '...|{'neg': 0.109, 'n...|          USA|
| 36|null| Before #coronavi...|   Frankapistan|       Nashville, TN|[('China', 'GPE')...|{'neg': 0.144, 'n...|          USA|
| 49|null| This week we are...|  JamesWithers3|  New York, New York|[('This week', 'D...|{'neg': 0.0, 'neu...|          USA|
|

In [39]:
twitter_df

DataFrame[_c0: int, geo: string, text: string, user: string, location: string, entities: string, sentiment: string, country: string]

In [40]:
twitter_df.first()

Row(_c0=0, geo=None, text='What is God saying to us about #coronavirus ?', user='petodinice', location='Lagos', entities="[('about #', 'CARDINAL')]", sentiment="{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'compound': 0.2732}", country='Nigeria')

In [41]:
twitter_df.take(5)

[Row(_c0=0, geo=None, text='What is God saying to us about #coronavirus ?', user='petodinice', location='Lagos', entities="[('about #', 'CARDINAL')]", sentiment="{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'compound': 0.2732}", country='Nigeria'),
 Row(_c0=1, geo=None, text='"BREAKING: ""this is disappointing', user=' but i took the test"". Arsenal\'s Mikel Arteta tests positive for #coronavirus', location='  -… "', entities='JerryfranksonJF', sentiment='Abuja, Nigeria', country='"[(""Arsenal\'s Mikel Arteta""'),
 Row(_c0=2, geo=None, text=' #Coronavirus testing must be made free to the public if we are going to understand the scope of this crisis. Anything le…', user='cek422', location='Pennsylvania, USA', entities='[]', sentiment="{'neg': 0.173, 'neu': 0.71, 'pos': 0.117, 'compound': -0.3767}", country='USA'),
 Row(_c0=3, geo=None, text=' Get ready for mass event crowd cancellations across the World starting this weekend: cricket in #Australia in empty st…', user='InfectiousDz', locatio

In [42]:
# RDD

In [43]:
x = spark.sparkContext.parallelize([1,4,8])
y = x.map(lambda x: (x, x*x))
print(x.collect())
print(y.collect())

[1, 4, 8]
[(1, 1), (4, 16), (8, 64)]


In [44]:
x = spark.sparkContext.parallelize([1,4,8])
y = x.flatMap(lambda x: (x, x*x))
print(x.collect())
print(y.collect())

[1, 4, 8]
[1, 1, 4, 16, 8, 64]


In [45]:
x

ParallelCollectionRDD[59] at readRDDFromFile at PythonRDD.scala:262

In [46]:
# TWITTER DATA

In [47]:
twitter_df.select('text').show(10, False)

+----------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                              |
+----------------------------------------------------------------------------------------------------------------------------------+
|What is God saying to us about #coronavirus ?                                                                                     |
|"BREAKING: ""this is disappointing                                                                                                |
| #Coronavirus testing must be made free to the public if we are going to understand the scope of this crisis. Anything le…        |
| Get ready for mass event crowd cancellations across the World starting this weekend: cricket in #Australia in empty st…          |
| The #coronavirus pandemic is revealing just how closely we are all 

In [48]:
twitter_df.select('user', 'text').show(10, False)

+------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+
|user                                                                          |text                                                                                                                              |
+------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+
|petodinice                                                                    |What is God saying to us about #coronavirus ?                                                                                     |
| but i took the test"". Arsenal's Mikel Arteta tests positive for #coronavirus|"BREAKING: ""this is disappointing                                      

In [49]:
# map and flatmap 

In [50]:
twitter_df.rdd.map(lambda line: line.text.split(" ")).take(5)

[['What', 'is', 'God', 'saying', 'to', 'us', 'about', '#coronavirus', '?'],
 ['"BREAKING:', '""this', 'is', 'disappointing'],
 ['',
  '#Coronavirus',
  'testing',
  'must',
  'be',
  'made',
  'free',
  'to',
  'the',
  'public',
  'if',
  'we',
  'are',
  'going',
  'to',
  'understand',
  'the',
  'scope',
  'of',
  'this',
  'crisis.',
  'Anything',
  'le…'],
 ['',
  'Get',
  'ready',
  'for',
  'mass',
  'event',
  'crowd',
  'cancellations',
  'across',
  'the',
  'World',
  'starting',
  'this',
  'weekend:',
  'cricket',
  'in',
  '#Australia',
  'in',
  'empty',
  'st…'],
 ['',
  'The',
  '#coronavirus',
  'pandemic',
  'is',
  'revealing',
  'just',
  'how',
  'closely',
  'we',
  'are',
  'all',
  'bound',
  'together...[A',
  'thread]',
  '']]

In [51]:
twitter_df.rdd.flatMap(lambda line: line.text.split(" ")).take(100)

['What',
 'is',
 'God',
 'saying',
 'to',
 'us',
 'about',
 '#coronavirus',
 '?',
 '"BREAKING:',
 '""this',
 'is',
 'disappointing',
 '',
 '#Coronavirus',
 'testing',
 'must',
 'be',
 'made',
 'free',
 'to',
 'the',
 'public',
 'if',
 'we',
 'are',
 'going',
 'to',
 'understand',
 'the',
 'scope',
 'of',
 'this',
 'crisis.',
 'Anything',
 'le…',
 '',
 'Get',
 'ready',
 'for',
 'mass',
 'event',
 'crowd',
 'cancellations',
 'across',
 'the',
 'World',
 'starting',
 'this',
 'weekend:',
 'cricket',
 'in',
 '#Australia',
 'in',
 'empty',
 'st…',
 '',
 'The',
 '#coronavirus',
 'pandemic',
 'is',
 'revealing',
 'just',
 'how',
 'closely',
 'we',
 'are',
 'all',
 'bound',
 'together...[A',
 'thread]',
 '',
 '',
 'COVID-19',
 'update',
 'as',
 'of',
 'this',
 'morning:1.',
 'Death',
 'toll',
 'in',
 'Italy',
 'passes',
 '1,0002.',
 "Arsenal's",
 'head',
 'coach',
 'Arteta',
 'tests',
 'positive3.',
 'US…',
 '',
 'It’s',
 'painful',
 'to',
 'say,',
 'but',
 'as',
 'an']

In [52]:
corona_df.filter("country='US'").show()

+---+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|_c0|           State|Country|               Lat|     Long|      Date|Confirmed|Death|Recovered|   state_cleaned|            City|
+---+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
| 98|      Washington|     US|           47.4009|-121.4905|2020-01-22|        0|    0|        0|      Washington|      Washington|
| 99|        New York|     US|           42.1657| -74.9481|2020-01-22|        0|    0|        0|        New York|        New York|
|100|      California|     US|           36.1162|-119.6816|2020-01-22|        0|    0|        0|      California|      California|
|101|   Massachusetts|     US|           42.2302| -71.5301|2020-01-22|        0|    0|        0|   Massachusetts|   Massachusetts|
|102|Diamond Princess|     US|           35.4437|  139.638|2020-01-22|        0|   

In [53]:
# Sorting 

In [54]:
corona_df.filter("country='US'").sort(col('Date'), ascending=False).show()

+-----+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|  _c0|           State|Country|               Lat|     Long|      Date|Confirmed|Death|Recovered|   state_cleaned|            City|
+-----+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|27764|      Washington|     US|           47.4009|-121.4905|2020-03-20|     1524|   83|        0|      Washington|      Washington|
|27784|         Arizona|     US|           33.7298|-111.4312|2020-03-20|       78|    0|        0|         Arizona|         Arizona|
|27765|        New York|     US|           42.1657| -74.9481|2020-03-20|     8310|   42|        0|        New York|        New York|
|27766|      California|     US|           36.1162|-119.6816|2020-03-20|     1177|   23|        0|      California|      California|
|27767|   Massachusetts|     US|           42.2302| -71.5301|2020-03-

In [55]:
corona_df.filter("country='US'").orderBy(col("Date"), ascending=False).show()

+-----+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|  _c0|           State|Country|               Lat|     Long|      Date|Confirmed|Death|Recovered|   state_cleaned|            City|
+-----+----------------+-------+------------------+---------+----------+---------+-----+---------+----------------+----------------+
|27764|      Washington|     US|           47.4009|-121.4905|2020-03-20|     1524|   83|        0|      Washington|      Washington|
|27784|         Arizona|     US|           33.7298|-111.4312|2020-03-20|       78|    0|        0|         Arizona|         Arizona|
|27765|        New York|     US|           42.1657| -74.9481|2020-03-20|     8310|   42|        0|        New York|        New York|
|27766|      California|     US|           36.1162|-119.6816|2020-03-20|     1177|   23|        0|      California|      California|
|27767|   Massachusetts|     US|           42.2302| -71.5301|2020-03-

In [56]:
# LIMITING DATA MOVEMENT --> AVOIDING WIDE TRANSFORMATION 

corona_df.filter("country='US'").sortWithinPartitions([col("Date"), col("Confirmed")], ascending=False).show()

+-----+--------------+-------+-------+---------+----------+---------+-----+---------+--------------+--------------+
|  _c0|         State|Country|    Lat|     Long|      Date|Confirmed|Death|Recovered| state_cleaned|          City|
+-----+--------------+-------+-------+---------+----------+---------+-----+---------+--------------+--------------+
|27765|      New York|     US|42.1657| -74.9481|2020-03-20|     8310|   42|        0|      New York|      New York|
|27764|    Washington|     US|47.4009|-121.4905|2020-03-20|     1524|   83|        0|    Washington|    Washington|
|27766|    California|     US|36.1162|-119.6816|2020-03-20|     1177|   23|        0|    California|    California|
|27773|    New Jersey|     US|40.2989|  -74.521|2020-03-20|      890|   11|        0|    New Jersey|    New Jersey|
|27776|      Illinois|     US|40.3495| -88.9861|2020-03-20|      585|    5|        0|      Illinois|      Illinois|
|27772|       Florida|     US|27.7663| -81.6868|2020-03-20|      563|   

In [57]:
corona_df.describe().show()

+-------+-----------------+---------+-----------+------------------+------------------+----------+------------------+------------------+------------------+-------------+-----------+
|summary|              _c0|    State|    Country|               Lat|              Long|      Date|         Confirmed|             Death|         Recovered|state_cleaned|       City|
+-------+-----------------+---------+-----------+------------------+------------------+----------+------------------+------------------+------------------+-------------+-----------+
|  count|            28143|    19116|      28143|             28143|             28143|     28143|             28143|             28143|             28143|        28143|      14573|
|   mean|          14071.0|     null|       null|30.965553459118834|-34.57031257861667|      null|161.88245744945456| 5.494368048893153| 60.17290267562094|         null|       null|
| stddev|8124.328649186959|     null|       null|19.365472826597646| 80.78375872452575|   

In [58]:
corona_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Date: string (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Death: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- state_cleaned: string (nullable = true)
 |-- City: string (nullable = true)



In [59]:
corona_df.filter('Confirmed>10000').sort(col('Confirmed'), ascending=False).show()

+-----+-----+-------+-------+--------+----------+---------+-----+---------+-------------+----+
|  _c0|State|Country|    Lat|    Long|      Date|Confirmed|Death|Recovered|state_cleaned|City|
+-----+-----+-------+-------+--------+----------+---------+-----+---------+-------------+----+
|27820|Hubei|  China|30.9756|112.2707|2020-03-20|    67800| 3133|    58382|        Hubei|null|
|27343|Hubei|  China|30.9756|112.2707|2020-03-19|    67800| 3130|    57682|        Hubei|null|
|26866|Hubei|  China|30.9756|112.2707|2020-03-18|    67800| 3122|    56927|        Hubei|null|
|26389|Hubei|  China|30.9756|112.2707|2020-03-17|    67799| 3111|    56003|        Hubei|null|
|25912|Hubei|  China|30.9756|112.2707|2020-03-16|    67798| 3099|    55142|        Hubei|null|
|25435|Hubei|  China|30.9756|112.2707|2020-03-15|    67794| 3085|    54288|        Hubei|null|
|24958|Hubei|  China|30.9756|112.2707|2020-03-14|    67790| 3075|    52960|        Hubei|null|
|24481|Hubei|  China|30.9756|112.2707|2020-03-13| 

In [60]:
corona_df.filter('Confirmed>10000').approxQuantile('Confirmed', [0.25, 0.5, 0.75, 0.9, 0.95], 0.9)

[10075.0, 10075.0, 10075.0, 10075.0, 67800.0]

In [61]:
# aggrigation function 

In [62]:
corona_df.agg({"Date":"max"}).collect()

[Row(max(Date)='2020-03-20')]

In [63]:
corona_df.agg({"Date":"max", "Confirmed":"max"}).collect()

[Row(max(Confirmed)=67800, max(Date)='2020-03-20')]

In [64]:
max_date = corona_df.agg({"Date":"max"})

In [65]:
max_date.show()

+----------+
| max(Date)|
+----------+
|2020-03-20|
+----------+



In [66]:
# Check the latest collected date 
import pyspark.sql.functions as F

corona_df.groupBy("Country", "State_cleaned").agg(F.max("Date").alias("Date")).show()

+--------------+--------------------+----------+
|       Country|       State_cleaned|      Date|
+--------------+--------------------+----------+
|      Cameroon|             Yaounde|2020-03-20|
|         China|             Qinghai|2020-03-20|
|        Cyprus|             Nicosia|2020-03-20|
|            US|            Michigan|2020-03-20|
|      Portugal|              Lisbon|2020-03-20|
|            US|            Colorado|2020-03-20|
|United Kingdom|      Cayman Islands|2020-03-20|
|         China|              Hainan|2020-03-20|
|            US|            Missouri|2020-03-20|
|     Australia|Australian Capita...|2020-03-20|
|            US|                Guam|2020-03-20|
|        France|             Reunion|2020-03-20|
|      Colombia|        Cundinamarca|2020-03-20|
|          Cuba|              Havana|2020-03-20|
|     Mauritius|          Port Louis|2020-03-20|
|       Ukraine|                Kiev|2020-03-20|
|         Benin|          Porto-Novo|2020-03-20|
|   Switzerland|    

In [67]:
corona_df.join(corona_df.groupBy("Country", "State_cleaned").agg(F.max("Date").alias("Date")), on=['Country', 'State_cleaned', 'Date'], how='inner').show()

+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+
|             Country|       state_cleaned|      Date|  _c0|           State|     Lat|     Long|Confirmed|Death|Recovered|City|
+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+
|            Thailand|             Bangkok|2020-03-20|27666|            null|    15.0|    101.0|      322|    1|       42|null|
|               Japan|             Hiraide|2020-03-20|27667|            null|    36.0|    138.0|      963|   33|      191|null|
|           Singapore|           Singapore|2020-03-20|27668|            null|  1.2833| 103.8333|      385|    0|      124|null|
|               Nepal|           Kathmandu|2020-03-20|27669|            null| 28.1667|    84.25|        1|    0|        1|null|
|            Malaysia|             Sarawak|2020-03-20|27670|            null|     2.5|    112.5|     103

In [68]:
# Aggregate, join and sort 

corona_df.join(corona_df.groupBy("Country", "State_cleaned").agg(F.max("Date").alias("Date")), on=['Country', 'State_cleaned', 'Date'], how='inner').sort("Confirmed", ascending=False).show()

+--------------+----------------+----------+-----+--------------+-------+-------------------+---------+-----+---------+----------+
|       Country|   state_cleaned|      Date|  _c0|         State|    Lat|               Long|Confirmed|Death|Recovered|      City|
+--------------+----------------+----------+-----+--------------+-------+-------------------+---------+-----+---------+----------+
|         China|           Hubei|2020-03-20|27820|         Hubei|30.9756|           112.2707|    67800| 3133|    58382|      null|
|         Italy|            Rome|2020-03-20|27682|          null|   43.0|               12.0|    47021| 4032|     4440|      null|
|         Spain|          Toledo|2020-03-20|27684|          null|   40.0|               -4.0|    20410| 1043|     1588|      null|
|       Germany|          Berlin|2020-03-20|27677|          null|   51.0|                9.0|    19848|   67|      180|      null|
|          Iran|          Tehran|2020-03-20|27821|          null|   32.0|          

In [73]:
corona_df.join(corona_df.groupBy("country", "state_cleaned").agg(F.max('Date').alias("Date")), on=['Country', 'state_cleaned', 'Date'], how='inner').sort("Confirmed", ascending=False).show(10)

+--------------+----------------+----------+-----+--------------+-------+-------------------+---------+-----+---------+--------+
|       Country|   state_cleaned|      Date|  _c0|         State|    Lat|               Long|Confirmed|Death|Recovered|    City|
+--------------+----------------+----------+-----+--------------+-------+-------------------+---------+-----+---------+--------+
|         China|           Hubei|2020-03-20|27820|         Hubei|30.9756|           112.2707|    67800| 3133|    58382|    null|
|         Italy|            Rome|2020-03-20|27682|          null|   43.0|               12.0|    47021| 4032|     4440|    null|
|         Spain|          Toledo|2020-03-20|27684|          null|   40.0|               -4.0|    20410| 1043|     1588|    null|
|       Germany|          Berlin|2020-03-20|27677|          null|   51.0|                9.0|    19848|   67|      180|    null|
|          Iran|          Tehran|2020-03-20|27821|          null|   32.0|               53.0|    

In [75]:
## WINDOW 

In [76]:
from pyspark.sql.functions import row_number 
from pyspark.sql.window import Window 

ws = Window().partitionBy("Country", "State_cleaned").orderBy(col("Date").desc())
corona_df.withColumn("row_num", row_number().over(ws)).show()

+-----+-----+--------+------------------+-------+----------+---------+-----+---------+-------------+----+-------+
|  _c0|State| Country|               Lat|   Long|      Date|Confirmed|Death|Recovered|state_cleaned|City|row_num|
+-----+-----+--------+------------------+-------+----------+---------+-----+---------+-------------+----+-------+
|27745| null|Cameroon|3.8480000000000003|11.5021|2020-03-20|       20|    0|        0|      Yaounde|null|      1|
|27268| null|Cameroon|3.8480000000000003|11.5021|2020-03-19|       13|    0|        0|      Yaounde|null|      2|
|26791| null|Cameroon|3.8480000000000003|11.5021|2020-03-18|       10|    0|        0|      Yaounde|null|      3|
|26314| null|Cameroon|3.8480000000000003|11.5021|2020-03-17|       10|    0|        0|      Yaounde|null|      4|
|25837| null|Cameroon|3.8480000000000003|11.5021|2020-03-16|        4|    0|        0|      Yaounde|null|      5|
|25360| null|Cameroon|3.8480000000000003|11.5021|2020-03-15|        2|    0|        0|  

In [77]:
corona_df.withColumn("row_num", row_number().over(ws)).where(col("row_num")==1).show()

+-----+--------------------+--------------+------------------+---------+----------+---------+-----+---------+--------------------+--------+-------+
|  _c0|               State|       Country|               Lat|     Long|      Date|Confirmed|Death|Recovered|       state_cleaned|    City|row_num|
+-----+--------------------+--------------+------------------+---------+----------+---------+-----+---------+--------------------+--------+-------+
|27745|                null|      Cameroon|3.8480000000000003|  11.5021|2020-03-20|       20|    0|        0|             Yaounde|    null|      1|
|27859|             Qinghai|         China|           35.7452|  95.9956|2020-03-20|       18|    0|       18|             Qinghai|    null|      1|
|27762|                null|        Cyprus|           35.1264|  33.4299|2020-03-20|       67|    0|        0|             Nicosia|    null|      1|
|27812|            Michigan|            US|           43.3266| -84.5361|2020-03-20|      552|    3|        0|   

In [78]:
corona_max_df = corona_df.join(corona_df.groupBy("Country", "State_cleaned").agg(F.max("Date").alias("Date")), on=['Country', 'State_cleaned', "Date"], how='inner')

In [79]:
corona_max_df.show()

+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+
|             Country|       state_cleaned|      Date|  _c0|           State|     Lat|     Long|Confirmed|Death|Recovered|City|
+--------------------+--------------------+----------+-----+----------------+--------+---------+---------+-----+---------+----+
|            Thailand|             Bangkok|2020-03-20|27666|            null|    15.0|    101.0|      322|    1|       42|null|
|               Japan|             Hiraide|2020-03-20|27667|            null|    36.0|    138.0|      963|   33|      191|null|
|           Singapore|           Singapore|2020-03-20|27668|            null|  1.2833| 103.8333|      385|    0|      124|null|
|               Nepal|           Kathmandu|2020-03-20|27669|            null| 28.1667|    84.25|        1|    0|        1|null|
|            Malaysia|             Sarawak|2020-03-20|27670|            null|     2.5|    112.5|     103

In [80]:
# Pivot 

In [81]:
corona_df.groupBy("Country").pivot("Date").agg(F.sum("Confirmed")).show()

+-----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|    Country|2020-01-22|2020-01-23|2020-01-24|2020-01-25|2020-01-26|2020-01-27|2020-01-28|2020-01-29|2020-01-30|2020-01-31|2020-02-01|2020-02-02|2020-02-03|2020-02-04|2020-02-05|2020-02-06|2020-02-07|2020-02-08|2020-02-09|2020-02-10|2020-02-11|2020-02-12|2020-02-13|2020-02-14|2020-02-15|2020-02-16|2020-02-17|2020-02-18|2020-02-19|2020-

In [82]:
corona_df.filter("Country=='US'").crosstab("State", "Date").show()

+-------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|         State_Date|2020-01-22|2020-01-23|2020-01-24|2020-01-25|2020-01-26|2020-01-27|2020-01-28|2020-01-29|2020-01-30|2020-01-31|2020-02-01|2020-02-02|2020-02-03|2020-02-04|2020-02-05|2020-02-06|2020-02-07|2020-02-08|2020-02-09|2020-02-10|2020-02-11|2020-02-12|2020-02-13|2020-02-14|2020-02-15|2020-02-16|2020-02-17|2020-02-18|

In [84]:
corona_max_df.groupBy("Country").agg(F.max('Confirmed').alias("Confirmed")).sort("Confirmed", ascending=False).show()

+--------------+---------+
|       Country|Confirmed|
+--------------+---------+
|         China|    67800|
|         Italy|    47021|
|         Spain|    20410|
|       Germany|    19848|
|          Iran|    19644|
|        France|    12612|
|  Korea, South|     8652|
|            US|     8310|
|   Switzerland|     5294|
|United Kingdom|     3983|
|   Netherlands|     2994|
|       Austria|     2388|
|       Belgium|     2257|
|        Norway|     1914|
|        Sweden|     1639|
|       Denmark|     1255|
|      Malaysia|     1030|
|      Portugal|     1020|
|         Japan|      963|
|       Czechia|      833|
+--------------+---------+
only showing top 20 rows



In [86]:
corona_df.groupBy("Country").agg(F.sum("Confirmed").alias("Confirmed"), F.sum("Recovered"), F.sum("Death")).orderBy("Confirmed", ascending=False).show()

+--------------+---------+--------------+----------+
|       Country|Confirmed|sum(Recovered)|sum(Death)|
+--------------+---------+--------------+----------+
|         China|  3287028|       1570179|    109614|
|         Italy|   326105|         30834|     23658|
|          Iran|   188884|         56432|      9883|
|  Korea, South|   155106|         10523|      1233|
|         Spain|   103290|          7093|      4198|
|       Germany|    89015|          1063|       217|
|        France|    71858|           342|      1623|
|            US|    67002|           249|      1135|
|   Switzerland|    25348|            96|       220|
|   Cruise Ship|    24092|          3544|       165|
|United Kingdom|    18816|           607|       584|
|   Netherlands|    16031|            14|       373|
|         Japan|    14471|          2279|       337|
|        Norway|    13891|            12|        32|
|        Sweden|    11847|            42|        58|
|       Belgium|    11707|            93|     

In [88]:
corona_df.filter("Country='Italy'").sort("Date", ascending=False).show()

+-----+-----+-------+----+----+----------+---------+-----+---------+-------------+----+
|  _c0|State|Country| Lat|Long|      Date|Confirmed|Death|Recovered|state_cleaned|City|
+-----+-----+-------+----+----+----------+---------+-----+---------+-------------+----+
|27682| null|  Italy|43.0|12.0|2020-03-20|    47021| 4032|     4440|         Rome|null|
|27205| null|  Italy|43.0|12.0|2020-03-19|    41035| 3405|     4440|         Rome|null|
|26728| null|  Italy|43.0|12.0|2020-03-18|    35713| 2978|     4025|         Rome|null|
|26251| null|  Italy|43.0|12.0|2020-03-17|    31506| 2503|     2941|         Rome|null|
|25774| null|  Italy|43.0|12.0|2020-03-16|    27980| 2158|     2749|         Rome|null|
|25297| null|  Italy|43.0|12.0|2020-03-15|    24747| 1809|     2335|         Rome|null|
|24820| null|  Italy|43.0|12.0|2020-03-14|    21157| 1441|     1966|         Rome|null|
|24343| null|  Italy|43.0|12.0|2020-03-13|    17660| 1266|     1439|         Rome|null|
|23866| null|  Italy|43.0|12.0|2

In [89]:
corona_max_df.withColumn("Active", corona_max_df.Confirmed - corona_max_df.Recovered - corona_max_df.Death).sort("Active", ascending=False).show()

+--------------+----------------+----------+-----+--------------+-------+-------------------+---------+-----+---------+----------+------+
|       Country|   state_cleaned|      Date|  _c0|         State|    Lat|               Long|Confirmed|Death|Recovered|      City|Active|
+--------------+----------------+----------+-----+--------------+-------+-------------------+---------+-----+---------+----------+------+
|         Italy|            Rome|2020-03-20|27682|          null|   43.0|               12.0|    47021| 4032|     4440|      null| 38549|
|       Germany|          Berlin|2020-03-20|27677|          null|   51.0|                9.0|    19848|   67|      180|      null| 19601|
|         Spain|          Toledo|2020-03-20|27684|          null|   40.0|               -4.0|    20410| 1043|     1588|      null| 17779|
|        France|          France|2020-03-20|27823|        France|46.2276|             2.2137|    12612|  450|       12|      null| 12150|
|          Iran|          Tehran|2

In [90]:
corona_max_df = corona_max_df.withColumn("Active", corona_max_df.Confirmed - corona_max_df.Recovered - corona_max_df.Death).sort("Active", ascending=False)

In [91]:
corona_max_df.show()

+--------------+----------------+----------+-----+--------------+-------+-------------------+---------+-----+---------+----------+------+
|       Country|   state_cleaned|      Date|  _c0|         State|    Lat|               Long|Confirmed|Death|Recovered|      City|Active|
+--------------+----------------+----------+-----+--------------+-------+-------------------+---------+-----+---------+----------+------+
|         Italy|            Rome|2020-03-20|27682|          null|   43.0|               12.0|    47021| 4032|     4440|      null| 38549|
|       Germany|          Berlin|2020-03-20|27677|          null|   51.0|                9.0|    19848|   67|      180|      null| 19601|
|         Spain|          Toledo|2020-03-20|27684|          null|   40.0|               -4.0|    20410| 1043|     1588|      null| 17779|
|        France|          France|2020-03-20|27823|        France|46.2276|             2.2137|    12612|  450|       12|      null| 12150|
|          Iran|          Tehran|2

In [92]:
corona_max_df.groupBy("Country").sum("Active").orderBy("sum(Active)", ascending=False).show()

+--------------+-----------+
|       Country|sum(Active)|
+--------------+-----------+
|         Italy|      38549|
|       Germany|      19601|
|            US|      18856|
|         Spain|      17779|
|        France|      12264|
|          Iran|      11466|
|  Korea, South|       7018|
|         China|       6731|
|   Switzerland|       5225|
|United Kingdom|       3769|
|   Netherlands|       2894|
|       Austria|       2373|
|       Belgium|       2219|
|        Norway|       1906|
|        Sweden|       1607|
|       Denmark|       1327|
|      Portugal|       1009|
|      Malaysia|        940|
|        Canada|        922|
|       Czechia|        829|
+--------------+-----------+
only showing top 20 rows



In [94]:
corona_max_df.select("Country", "State_cleaned", "Confirmed", "Recovered").filter(col("Country").isin("Australia", "China")).groupBy("Country").sum().show(100)

+---------+--------------+--------------+
|  Country|sum(Confirmed)|sum(Recovered)|
+---------+--------------+--------------+
|    China|         81250|         71266|
|Australia|           791|            26|
+---------+--------------+--------------+



In [95]:
corona_max_df.select("Country", "State_cleaned", "Confirmed", "Recovered").filter(col("Country").isin("Australia", "China")).cube("Country").sum().show(100)

+---------+--------------+--------------+
|  Country|sum(Confirmed)|sum(Recovered)|
+---------+--------------+--------------+
|Australia|           791|            26|
|     null|         82041|         71292|
|    China|         81250|         71266|
+---------+--------------+--------------+



In [97]:
corona_max_df.select("Country", "State_cleaned", "Confirmed", "Recovered").filter(col("Country").isin("Australia", "China")).cube("Country", 'State_cleaned').sum().show(100)

+---------+--------------------+--------------+--------------+
|  Country|       State_cleaned|sum(Confirmed)|sum(Recovered)|
+---------+--------------------+--------------+--------------+
|     null|           Guangdong|          1395|          1323|
|     null|               Hunan|          1018|          1014|
|     null|               Gansu|           134|            98|
|     null|               Hebei|           318|           310|
|     null|               Jilin|            93|            92|
|     null|          Queensland|           184|             8|
|    China|            Shandong|           762|           748|
|     null|  Northern Territory|             3|             0|
|    China|           Chongqing|           576|           570|
|     null|            Liaoning|           126|           122|
|     null|             Beijing|           491|           390|
|    China|             Ningxia|            75|            75|
|    China|                null|         81250|        

In [98]:
# rollup
corona_max_df.select("Country", "State_cleaned", "Confirmed", "Recovered").filter(col("Country").isin("Australia", "China")).rollup("Country", 'State_cleaned').sum().show(100)

+---------+--------------------+--------------+--------------+
|  Country|       State_cleaned|sum(Confirmed)|sum(Recovered)|
+---------+--------------------+--------------+--------------+
|    China|            Shandong|           762|           748|
|    China|           Chongqing|           576|           570|
|    China|             Ningxia|            75|            75|
|    China|                null|         81250|         71266|
|Australia|  Northern Territory|             3|             0|
|    China|            Shanghai|           371|           326|
|    China|               Anhui|           990|           984|
|    China|             Qinghai|            18|            18|
|    China|           Hong Kong|           256|            98|
|    China|              Shanxi|           133|           133|
|    China|              Hainan|           168|           161|
|     null|                null|         82041|         71292|
|    China|             Guangxi|           254|        

In [99]:
# Correlation 

corona_max_df.corr('Confirmed', 'Recovered')

0.8052934236742177

In [100]:
# Caching data in memory 
corona_max_df.cache()

DataFrame[Country: string, state_cleaned: string, Date: string, _c0: int, State: string, Lat: double, Long: double, Confirmed: int, Death: int, Recovered: int, City: string, Active: int]

In [102]:
%timeit corona_max_df.count()

The slowest run took 6.50 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 474 ms per loop


In [105]:
from pyspark import StorageLevel 

corona_max_df.persist(StorageLevel.MEMORY_AND_DISK)

DataFrame[Country: string, state_cleaned: string, Date: string, _c0: int, State: string, Lat: double, Long: double, Confirmed: int, Death: int, Recovered: int, City: string, Active: int]

In [106]:
%timeit corona_max_df.count()

1 loop, best of 5: 425 ms per loop


In [107]:
# Pandas 

In [108]:
pd = corona_df.toPandas()

In [109]:
pd.corr()

,_c0,Lat,Long,Confirmed,Death,Recovered
_c0,1.000000,-0.001594,-0.005303,0.046789,0.043155,0.042857
Lat,-0.001594,1.000000,-0.383389,0.008531,0.007041,0.000778
Long,-0.005303,-0.383389,1.000000,0.098893,0.074918,0.079047
Confirmed,0.046789,0.008531,0.098893,1.000000,0.963376,0.853924
Death,0.043155,0.007041,0.074918,0.963376,1.000000,0.857292
Recovered,0.042857,0.000778,0.079047,0.853924,0.857292,1.000000


In [110]:
corona_max_df.createOrReplaceTempView("corona")

In [111]:
spark.sql("select * from corona").show()

+--------------+----------------+----------+-----+--------------+-------+-------------------+---------+-----+---------+----------+------+
|       Country|   state_cleaned|      Date|  _c0|         State|    Lat|               Long|Confirmed|Death|Recovered|      City|Active|
+--------------+----------------+----------+-----+--------------+-------+-------------------+---------+-----+---------+----------+------+
|         Italy|            Rome|2020-03-20|27682|          null|   43.0|               12.0|    47021| 4032|     4440|      null| 38549|
|       Germany|          Berlin|2020-03-20|27677|          null|   51.0|                9.0|    19848|   67|      180|      null| 19601|
|         Spain|          Toledo|2020-03-20|27684|          null|   40.0|               -4.0|    20410| 1043|     1588|      null| 17779|
|        France|          France|2020-03-20|27823|        France|46.2276|             2.2137|    12612|  450|       12|      null| 12150|
|          Iran|          Tehran|2

In [112]:
spark.sql("select * from corona where Country in ('Australia', 'China') order by Country").show()

+---------+--------------------+----------+-----+--------------------+------------------+------------------+---------+-----+---------+----+------+
|  Country|       state_cleaned|      Date|  _c0|               State|               Lat|              Long|Confirmed|Death|Recovered|City|Active|
+---------+--------------------+----------+-----+--------------------+------------------+------------------+---------+-----+---------+----+------+
|Australia|          Queensland|2020-03-20|27674|          Queensland|          -28.0167|             153.4|      184|    0|        8|null|   176|
|Australia|Australian Capita...|2020-03-20|28068|Australian Capita...|          -35.4735|          149.0124|        6|    0|        0|null|     6|
|Australia|   Western Australia|2020-03-20|27715|   Western Australia|          -31.9505|          115.8605|       64|    1|        0|null|    63|
|Australia|     New South Wales|2020-03-20|27672|     New South Wales|          -33.8688|          151.2093|      353|

In [115]:
# Roll up 

spark.sql("select Country, State_cleaned, SUM(Confirmed), SUM(Recovered) from corona where Country in ('Australia', 'Canada') GROUP BY ROLLUP (Country, State_cleaned) order by Country").show()

+---------+--------------------+--------------+--------------+
|  Country|       State_cleaned|sum(Confirmed)|sum(Recovered)|
+---------+--------------------+--------------+--------------+
|     null|                null|          1734|            35|
|Australia|          Queensland|           184|             8|
|Australia|     New South Wales|           353|             4|
|Australia|Australian Capita...|             6|             0|
|Australia|   Western Australia|            64|             0|
|Australia|                null|           791|            26|
|Australia|  Northern Territory|             3|             0|
|Australia|            Victoria|           121|             8|
|Australia|            Tasmania|            10|             3|
|Australia|From Diamond Prin...|             0|             0|
|Australia|     South Australia|            50|             3|
|   Canada|       New Brunswick|            11|             0|
|   Canada|        Saskatchewan|            20|        

In [116]:
# Roll up 

spark.sql("select Country, State_cleaned, SUM(Confirmed), SUM(Recovered) from corona where Country in ('Australia', 'Canada') GROUP BY CUBE (Country, State_cleaned) order by Country").show()

+-------+--------------------+--------------+--------------+
|Country|       State_cleaned|sum(Confirmed)|sum(Recovered)|
+-------+--------------------+--------------+--------------+
|   null|       New Brunswick|            11|             0|
|   null|     New South Wales|           353|             4|
|   null|            Victoria|           121|             8|
|   null|         Nova Scotia|            15|             0|
|   null|Australian Capita...|             6|             0|
|   null|          Queensland|           184|             8|
|   null|     South Australia|            50|             3|
|   null|      Grand Princess|            10|             0|
|   null|        Saskatchewan|            20|             0|
|   null|Newfoundland and ...|             4|             0|
|   null|   Western Australia|            64|             0|
|   null|            Tasmania|            10|             3|
|   null|    British Columbia|           271|             4|
|   null|             On